ELEC-E5550 - Statistical Natural Language Processing
# SET 7: Machine Translation Evaluation

# Released: 05.03.2024
# Deadline: 15.03.2024 at midnight

In this assignment, you will learn how machine translation systems can be automatically evaluated, using the BLEU score.

KEYWORDS:
* BLEU

Do not use external libraries in this assignment. Python standard library is allowed.

# Table of contents

* [TASK 1: BLEU score](#task1)
    * [Step 1.1: Match N-grams](#subtask1_1)
    * [Step 1.2: Match lengths](#subtask1_2)
    * [Step 1.3: Put it together](#subtask1_3)
    * [Step 1.4: Reflect on BLEU](#subtask1_4)

Evaluation is a particularly difficult topic in machine translation. Human evaluations are preferred, but expensive. There are multiple automatic evaluation metrics in use. The most common one is BLEU. In this assignment you will implement BLEU calculation - it is computationally very simple.

## TASK 1 <a class="anchor" id="task1"></a>
## BLEU score

**BLEU**, or bilingual evaluation understudy, was proposed in 2002, here: https://www.aclweb.org/anthology/P02-1040.pdf. Take read through the paper, it is clearly written and provides not just the algorithm, but also context and reasons for the design choices. We will implement the algorithm as described by the paper, so you can refer to it.

### High-level overview

The essential requirement for BLEU is an annotated test corpus. The test corpus consists of source language segments and multiple (or at least one) reference translations for each segment. The reference translations are made by human translators. Multiple reference translations are used to account for the fact that there is no single correct translation.

A machine translation system produces a hypothesis translation for each source segment. The hypothesis is then compared to all the references. BLEU looks for matching N-grams of different lengths N. The more matches, the better. Additionally, BLEU compares the length of the hypothesis with the lengths of the translations - brevity is penalized. These two components are combined into a score between 0 and 1.

## 1.1 Match N-grams (3 Points) <a class="anchor" id="subtask1_1"></a>
Specifically, BLEU uses a concept called modified precision. The hypothesis n-grams can find a match in any reference - but if the same n-gram occurs multiple times in the hypothesis, the number of matches is clipped to the maximum number of times the n-gram occurs in any single reference. The intuition is simple - the references are likely to have many words and phrases in common. Without the clipping, repeating the same likely words in the hypothesis would yield an inflated BLEU score. The hypothesis "the the the the the the the" would find many matches from almost any set of references.

In [1]:
"""
You may use this ngram forming implementation in your solution.
This returns a generator. If you absolutely need a list, you can
simply call
>>> list(form_ngrams(...))
"""
from collections import deque

def form_ngrams(tokens, n):
    """Forms all ngrams from tokens
    
    Arguments
    ---------
    tokens : iterable
        Tokens to form n-grams from
    n : int
        The length of ngrams to form
    
    Yields
    ------
    tuple
        Yields each ngram as a tuple of tokens.   
    """
    window = deque(maxlen=n)
    for i, token in enumerate(tokens, start=1):
        window.append(token)
        if i >= n:
            yield tuple(window)
            
print(list(form_ngrams(["furiously", "sleep", "ideas", "green", "colorless"], 3)))

[('furiously', 'sleep', 'ideas'), ('sleep', 'ideas', 'green'), ('ideas', 'green', 'colorless')]


In [2]:
from collections import Counter

def modified_precision(hypothesis, references, n):
    """Matches hypothesis N-grams to references at given length N
    
    Hypothesis and references are given as plain token sequences. This function
    forms N-grams of appropriate lengths.
    
    Count the number of times each N-gram occurs in the hypothesis.
    Then take the union of N-gram counts in all the references:
    for each N-gram, keep track of the maximum number of times it occurs in a single
    reference. This forms the reference counts.
    Finally, count the clipped hits - sum each N-gram count in the hypothesis,
    but clip those hypothesis counts to the reference counts.
    
    This returns the hits and number of candidates separately, since those values
    are aggregated over the full corpus in BLEU.
    The modified precision for this single segment would be clipped_hits / num_candidates
    
    Arguments
    ---------
    hypothesis : list of str
        List of tokens - the hypothesis translation.
        E.G. ["alo", "mundo"]
    references : list of lists of str
        List of references - and each reference is a list of tokens.
        E.G. [["hola", "mundo"], ["alo", "tierra"]]
    
    Returns
    -------
    int
        The number of candidate N-grams of length N in the hypothesis that found
        a match.
    int
        The total number of candidate N-grams of length N in the hypothesis.
    """
    hyp_counts = Counter(form_ngrams(hypothesis,n))
    num_candidates = sum(hyp_counts.values())
    if num_candidates == 0:
        # If hypothesis is shorter than N, we get here.
        # We return 0, 1 instead of 0, 0, so that we don't
        # run into divide-by-zero errors.
        return 0, 1
    
    # YOUR CODE HERE
    # raise NotImplementedError()

    # Initialize reference counts for maximum occurrences of each n-gram across all references
    max_ref_ngram_counts = Counter()
    
    # Calculate maximum n-gram counts across all references
    for reference in references:
        ref_ngrams = Counter(form_ngrams(reference, n))
        for ngram in ref_ngrams:
            max_ref_ngram_counts[ngram] = max(max_ref_ngram_counts.get(ngram, 0), ref_ngrams[ngram])
    
    # Calculate clipped hits
    clipped_hits = 0
    for ngram, count in hyp_counts.items():
        clipped_hits += min(count, max_ref_ngram_counts.get(ngram, 0))
    
    # Return the numerator and denominator separately, since
    # they get aggregated over the whole corpus.
    return clipped_hits, num_candidates

def form_ngrams(tokens, n):
    for i in range(len(tokens) - n + 1):
        yield tuple(tokens[i:i+n])

In [3]:
from nose.tools import assert_equal

# Basic sanity check:
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","d"]], 1), (4, 4))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","d"]], 2), (3, 3))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","d"]], 3), (2, 2))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","d"]], 4), (1, 1))

# Small difference:
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"]], 1), (3, 4))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"]], 2), (2, 3))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"]], 3), (1, 2))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"]], 4), (0, 1))

# More references:
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"],["b","c","c","d"]], 1), (4, 4))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"],["b","c","c","d"]], 2), (3, 3))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"],["b","c","c","d"]], 3), (1, 2))
assert_equal(modified_precision(["a","b","c","d"], [["a","b","c","e"],["b","c","c","d"]], 4), (0, 1))

# Clipping:
assert_equal(modified_precision(["c","c","c","c"], [["a","b","c","e"],["b","c","c","d"]], 1), (2, 4))
assert_equal(modified_precision(["c","c","c","c"], [["a","b","c","e"],["b","c","c","d"]], 2), (1, 3))
assert_equal(modified_precision(["c","c","c","c"], [["a","b","c","e"],["b","c","c","d"]], 3), (0, 2))
assert_equal(modified_precision(["c","c","c","c"], [["a","b","c","e"],["b","c","c","d"]], 4), (0, 1))

# With real text:
assert_equal(modified_precision(["waters", "destroyed", "perfume"], 
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                1), (2, 3))
assert_equal(modified_precision(["waters", "destroyed", "perfume"], 
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                2), (0, 2))

austen_hypo = ["it", "is", "a", "widely", "accepted", "truth", 
              "that", "an", "unmarried", "wealthy", "man", 
              "necessarily", "needs", "a", "wife", "alongside", "him"]
austen_ref = [["it", "is", "a", "truth", "universally", "acknowledged",
               "that", "a", "single", "man", "in", "posession", "of","a""good","fortune",
               "must", "be", "in", "want", "of", "a", "wife"],
              ["everybody", "knows", "the","rich","man","necessarily", "needs", "a", "wife"]]
assert_equal(modified_precision(austen_hypo, austen_ref, 3), (4,15))
assert_equal(modified_precision(austen_hypo, austen_ref, 4), (2,14))
assert_equal(modified_precision(austen_hypo, austen_ref, 5), (1,13))
assert_equal(modified_precision(austen_hypo, austen_ref, 6), (0,12))

assert_equal(modified_precision(["waters", "destroyed", "perfume"],
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                1), (2, 3))
assert_equal(modified_precision(["waters", "destroyed", "perfume"],
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                2), (0, 2))
assert_equal(modified_precision(["waters", "destroyed", "perfume"],
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                3), (0, 1))
assert_equal(modified_precision(["waters", "destroyed", "perfume"],
                                [["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
                                4), (0, 1))

assert_equal(modified_precision(["the", "the", "the", "the", "the", "the", "the"],
                                [["the", "cat", "is", "on", "the", "mat"], 
                                 ["there", "is", "a", "cat", "on", "the", "mat"]], 1), (2, 7))
assert_equal(modified_precision(["the", "the", "the", "the", "the", "the", "the"],
                                [["the", "cat", "is", "on", "the", "mat"], 
                                 ["there", "is", "a", "cat", "on", "the", "mat"]], 2), (0, 6))
assert_equal(modified_precision(["the", "the", "the", "the", "the", "the", "the"],
                                [["the", "cat", "is", "on", "the", "mat"], 
                                 ["there", "is", "a", "cat", "on", "the", "mat"]], 3), (0, 5))
assert_equal(modified_precision(["the", "the", "the", "the", "the", "the", "the"],
                                [["the", "cat", "is", "on", "the", "mat"], 
                                 ["there", "is", "a", "cat", "on", "the", "mat"]], 4), (0, 4))



## 1.2 Match lengths (1 Point) <a class="anchor" id="subtask1_2"></a>

BLEU penalizes short hypotheses. This counter part is needed for precision, because precision by itself encourages adding only the most certain candidates. Typically recall is used as precision's pair, why not here? You can read the original paper to see. Instead of recall, BLEU uses a Brevity Penalty, which is computed over the whole test corpus. Here we just find the values which are aggregated - the hypothesis length and the closest reference length.

In [4]:
def brevity_penalty_values(hypothesis, references):
    """Computes the length of the hypothesis and find the closest reference length.
    
    Pick the shortest reference length if there are many equally close
    
    Arguments
    ---------
    hypothesis : list of str
        List of tokens - the hypothesis translation.
        E.G. ["alo", "mundo"]
    references : list of lists of str
        List of references - and each reference is a list of tokens.
        E.G. [["hola", "mundo"], ["alo", "tierra"]]
        
    Returns
    -------
    int
        The length of the hypothesis
    int
        The length of reference, which is closest to the hypothesis in length
        
    Note
    ----
    This just considers length in tokens - this doesn't consider N-grams.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    hyplen = len(hypothesis)
    closest_reflen = min(references, key=lambda ref: (abs(len(ref) - hyplen), len(ref)))
    closest_reflen = len(closest_reflen)
    
    return hyplen, closest_reflen

In [5]:
from nose.tools import assert_equal

# Basic cases:
assert_equal(brevity_penalty_values(["a","b","c"], [["e","f","g"],["a","b"]]), (3,3))
assert_equal(brevity_penalty_values(["a","b","c"], [["e","f","g","h"],["a","b"]]), (3,2))
assert_equal(brevity_penalty_values(["a","b","c","d"], [["e","f","g","h","i","j"],["a","b"]]), (4,2))
assert_equal(brevity_penalty_values(["a","b","c","d"], [["e","f","g","h"],["a","b"]]), (4,4))
assert_equal(brevity_penalty_values(["a","b"], [["e","f","g","h"],["a","b","c","d","e"]]), (2,4))
assert_equal(brevity_penalty_values([], [["a","b","c","d","e"],[]]), (0,0))
assert_equal(brevity_penalty_values(["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q"],
                                    [["r","s","t","u","v","x","y"],
                                     ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v"]]), 
             (17, 22))




## 1.3 Put it together <a class="anchor" id="subtask1_3"></a>

The key in BLEU is to aggregate the statistics from above over a large test corpus. BLEU computed on an individual sentence does not correlate well with human scores. Even if this sentence-wise BLEU is computed for all individual sentences in a test corpus and then averaged, the resulting metric is much less useful than when a single BLEU score is computed over the full corpus.

The modified precision aggregation happens at each N-gram length separately. Typically, BLEU-4 is computed, meaning maximum N-gram length 4. In that case, 8 precision values are collected: total number of hits and total number of candidates at each each N.

The length values just consider length in tokens, so only two values are needed: the total length of the hypotheses and the total length of the best matching reference lengths. The brevity penalty (BP) is then computed as follows:
$$
BP = 1,~c>r
$$
$$
BP = exp(1-\dfrac{r}{c}), c\leq r
$$
with aggregated hypothesis length $c$ and aggregated closest reference length $r$.

The metric is finally computed by:
$$
BLEU = BP\times exp\left( \sum_n^N \left[\dfrac{1}{N}log(p_n)\right]\right)
$$

Here aggregated modified precision at N-gram length $n$ is simply computed as: $p_n = \dfrac{\text{Clipped Hits}}{\text{Total Candidates}} $


In [6]:
# Use these exp and log operations:
from math import exp, log
EPSILON = 1e-10  # This is used for smoothing zero counts (already implemented)

def BLEU(hypothesis_set, references_set, max_n=4):
    """Computes BLEU over a corpus
    
    Use your two earlier functions.
    """

    # Collect these statistics over the corpus ...
    hits_totals = {N: 0 for N in range(1,max_n+1)} # Hits by N-gram N
    num_candidates_totals = {N: 0 for N in range(1,max_n+1)} # Num candidates by N
    hyplen_total = 0
    closest_reflen_total = 0
    # ... in the following loop:
    for hypothesis, references in zip(hypothesis_set, references_set):
        # YOUR CODE HERE 
        # raise NotImplementedError()
        
        hyplen, closest_reflen = brevity_penalty_values(hypothesis, references)
        hyplen_total += hyplen
        closest_reflen_total += closest_reflen

        for N in range(1, max_n+1):
            hits, num_candidates = modified_precision(hypothesis, references, N)
            hits_totals[N] += hits
            num_candidates_totals[N] += num_candidates
            
    # Additionally in BLEU, substituting a low value for zero counts is used
    # to avoid log(0) problems.
    # Smooth zero counts:
    hits_totals = {N: hits if hits > 0 else EPSILON for N, hits in hits_totals.items()}
    
    # Now, compute brevity penalty based on 
    # hyplen_total and closest_reflen_totals
    # Corner case: if hyplen_total == 0: brevity_penalty = 0.
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    if hyplen_total == 0:
        brevity_penalty = 0
    else:
        brevity_penalty = exp(1 - closest_reflen_total / hyplen_total) if hyplen_total < closest_reflen_total else 1

        
    # Finally compute the precisions, and weight them uniformly by 1 / max_n
    # Then multiply by brevity penalty and return the final score.
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    log_precision_sum = sum((log(hits_totals[N] / num_candidates_totals[N]) for N in range(1, max_n+1))) / max_n
    BLEU_score = brevity_penalty * exp(log_precision_sum)
    return BLEU_score

In [7]:
from numpy.testing import assert_almost_equal
from nose.tools import assert_equal

# Basic corpus input:
hyps = [["a", "b", "c","d"],
        ["colourless", "green", "ideas"],
       ["a","c","d"]]        
references = [[["a","b","c"], ["a","b","c","d","e"]], 
              [["ideas", "green", "colourless", "sleep"],["colourless", "green"]],
              [["a","b","c"], ["a","b","c","d","e"]]]
assert_almost_equal(BLEU(hyps, references), 0.5873949094699213, 2)
# More corpus input:
hyps = [["waters", "destroyed", "perfume"],
        ["the", "the", "the", "the", "the", "the", "the"],
        ["it", "is", "a", "widely", "accepted", "truth", 
         "that", "an", "unmarried", "wealthy", "man", 
         "necessarily", "needs", "a", "wife", "alongside", "him"]]        
references = [[["the", "city", "of", "cologne", "was", "destroyed", "by", "flood", "waters"]], 
              [["the", "cat", "is", "on", "the", "mat"], ["there", "is", "a", "cat", "on", "the", "mat"]],
              [["it","is","a","truth","universally","acknowledged",
                "that", "a", "single", "man", "in", "posession", "of","a""good","fortune",
               "must","be","in","want","of","a","wife"],
               ["everybody", "knows", "the","rich","man","necessarily", "needs", "a", "wife"]]]
assert_almost_equal(BLEU(hyps, references), 0.1502348037292212, 2)
# Exactly the same:
assert_equal(BLEU([[1,2,3,4]], [[[1,2,3,4]]]), 1.0)
# Slightly different:
assert_almost_equal(BLEU([[5,1,2,3,4]], [[[1,2,3,4,5]]]), 0.7071067811865475, 2)
# No hypothesis:
assert_equal(BLEU([[]], [[["Silence"]]]), 0.)
# No hits at N=4:
assert_almost_equal(BLEU([[1,2,3,4]], [[[1,2,3,5]]]), 0.0022360679774997894, 2)





## 1.4 Reflect on BLEU <a class="anchor" id="subtask1_4"></a>
Briefly answer the following questions:

(Each answer should be **no longer than 5 sentences and must not exceed 150 words**. Be aware that answers surpassing these limits may result in a deduction of points.)

1. Will providing more references increase the BLEU score? Why?

2. Why is the brevity penalty is used instead of the more traditional recall?

3. Will a human translator always get a BLEU score of 1? Why?

4. Autograding these written questions is impossible. Or is it? How could we use BLEU to autograde these answers? The upside would be to save a lot of teaching assistant time, but what would be some downsides?

**1. Will providing more references increase the BLEU score? Why?**

Yes, providing more references can increase the BLEU score. This is because BLEU measures accuracy by comparing the n-gram matches between the references and the translation (or hypothesis). Higher n-gram accuracy scores means it is likely that different formulations of the same concept found in the hypothesis would find a match in one of the references when there are more references. More references can also help us find a reference length closer to the hypothesis length and thus lowers the brevity penalty. However, after a certain number of references is added , the BLUE score improvement may reach a plateau and it no longer increases.

**2. Why is the brevity penalty is used instead of the more traditional recall?**

The brevity penalty is used in BLEU instead of recall to deal with the issue of translation extreme brevity. Recall measures how many of the correct n-grams are in the hypothesis from all possible correct n-grams from the references, which could favor longer translations that cover more reference n-grams, regardless of their relevance or correctness. However, BLEU tries to evaluate the quality of machine translations by penalizing very short translations. The brevity penalty ensures that translations are not only precise but also closer in length to the reference translations. 

=> Brevity penalty is used instead of recall to ensure machine translations are not too short, nor too long due to the recall metrics that prefers longer translations.  

**3. Will a human translator always get a BLEU score of 1? Why?**

No, a human translator will not always get a BLEU score of 1. The BLEU score measures the similarity between a machine-generated translation and a referenced translation by humans based on the n-grams count. However, even high-quality human translations can choose different wording or expressions that do not match those in the reference translations, leading to less than perfect n-gram overlapping. Additionally, the BLEU score has a brevity penalty that penalizes translations for being too short no matter their quality. Since there is often more than one valid way to translate a text, another human translated version might differ from the reference version in length and word choices, thus not achieving a BLEU score of 1.

**4. Autograding these written questions is impossible. Or is it? How could we use BLEU to autograde these answers? The upside would be to save a lot of teaching assistant time, but what would be some downsides?**

- Autograding written questions with BLEU is possible but there are some significant downsides when we use it. 

- For BLEU score to be applied, it should compare student answers to high-quality reference answers produced by the course staff, which will reduce grading time by the TAs. 

- The key downside is BLEU's focus on exact n-gram matches, which may not accurately contain the original expression in student responses. This method will unfortunately penalize varied but still otherwise valid answers that do not match the reference wording closely.

- Another downside is that BLEU cannot assess the correctness of content aside from lexical similarity, meaning it might miss a serious misconception in student answers that are somehow phrased similarly to the reference.